In [1]:
import numpy as np
import pygsti
from matplotlib import pyplot as plt
from pprint import pprint

In [2]:
import pygsti.extras.interpygate as interp
from pygsti.tools.basistools import change_basis

In [3]:
np.set_printoptions(precision=4, linewidth=120, floatmode='maxprec_equal')

## Build model gate

In [4]:
sigI = np.array([[1.,0],[0, 1]], dtype='complex')
sigX = np.array([[0, 1],[1, 0]], dtype='complex')
sigY = np.array([[0,-1],[1, 0]], dtype='complex') * 1.j
sigZ = np.array([[1, 0],[0,-1]], dtype='complex')
sigM = (sigX - 1.j*sigY)/2.
sigP = (sigX + 1.j*sigY)/2.

In [5]:
class SingleQubitTargetOp(pygsti.modelmembers.operations.OpFactory):

    def __init__(self):
        self.process = self.create_target_gate
        pygsti.modelmembers.operations.OpFactory.__init__(self, 4, evotype="densitymx")
        self.dim = 4

    def create_target_gate(self, v):
        
        phi, theta = v
        target_unitary = (np.cos(theta/2) * sigI + 
                          1.j * np.sin(theta/2) * (np.cos(phi) * sigX + np.sin(phi) * sigY))
        superop = change_basis(np.kron(target_unitary.conj(), target_unitary), 'col', 'pp')

        return superop
    
    def create_object(self, args=None, sslbls=None):
        assert(sslbls is None)
        mx = self.process([*args])
        return pygsti.modelmembers.operations.StaticArbitraryOp(mx)

In [6]:
class SingleQubitGate(interp.PhysicalProcess):
    def __init__(self, 
                 verbose=False,
                 cont_param_gate = False,
                 num_params = None,
#                  process_shape = (4, 4),
                 item_shape = (4,4),
                 aux_shape = None,
                 num_params_evaluated_as_group = 0,
                 ):

        self.verbose = verbose

        self.cont_param_gate = cont_param_gate

        self.num_params = num_params
        self.item_shape = item_shape

        self.aux_shape = aux_shape
        self.num_params_evaluated_as_group = num_params_evaluated_as_group

   
    def create_process_matrix(self, v, comm=None, return_generator=False):                                                                                                                                                                                             

        processes = []
        phi, theta, t = v
        theta = theta * t
        target_unitary = (np.cos(theta/2) * sigI + 
                          1.j * np.sin(theta/2) * (np.cos(phi) * sigX + np.sin(phi) * sigY))
        superop = change_basis(np.kron(target_unitary.conj(), target_unitary), 'col', 'pp')
        processes += [superop]
        return np.array(processes) if (processes is not None) else None

    def create_aux_info(self, v, comm=None):
        return []  # matches aux_shape=() above
    
    def create_process_matrices(self, v, grouped_v, comm=None):
        assert(len(grouped_v) == 1)  # we expect a single "grouped" parameter

        processes = []
        times = grouped_v[0]
        phi_in, theta_in = v
        for t in times:
            phi = phi_in
            theta = theta_in * t
            target_unitary = (np.cos(theta/2) * sigI + 
                              1.j * np.sin(theta/2) * (np.cos(phi) * sigX + np.sin(phi) * sigY))
            superop = change_basis(np.kron(target_unitary.conj(), target_unitary), 'col', 'pp')
            processes += [superop]
        return np.array(processes) if (processes is not None) else None

    def create_aux_infos(self, v, grouped_v, comm=None):
        import numpy as np
        times = grouped_v[0]
        return [ [] for t in times] # list elements must match aux_shape=() above

In [7]:
param_ranges = [(0.9,1.1,3)]

arg_ranges = [2*np.pi*(1+np.cos(np.linspace(np.pi,0, 7)))/2,
# arg_ranges = [(0,2*np.pi,7),
              (0, np.pi, 3)] 
arg_indices = [0,1]


target_op = SingleQubitTargetOp()
gate_process = SingleQubitGate(num_params = 3,num_params_evaluated_as_group = 1)

In [8]:
opfactory_linear = interp.InterpolatedOpFactory.create_by_interpolating_physical_process(
    target_op, gate_process, argument_ranges=arg_ranges, parameter_ranges=param_ranges, 
    argument_indices=arg_indices, interpolator_and_args='linear')

opfactory_spline = interp.InterpolatedOpFactory.create_by_interpolating_physical_process(
    target_op, gate_process, argument_ranges=arg_ranges, parameter_ranges=param_ranges, 
    argument_indices=arg_indices, interpolator_and_args='spline')


### Check that the interpolator is working

In [9]:
if False:
    indices = (2,3)
    nparams = 30

    x = np.linspace(0,2*np.pi, nparams)
    y = np.linspace(0, np.pi, nparams)
    for p in np.linspace(.9,1.1,5):

        def interp_linear(x, y):    
            op = opfactory_linear.create_op([x, y])
            return op.base_interpolator([x,y,p])[indices]

        def interp_spline(x, y):    
            op = opfactory_spline.create_op([x, y])
            return op.base_interpolator([x,y,p])[indices]

        def truth(x, y):
            tmp_gate = gate_process.create_process_matrix([x,y,p])[0]
            tar_gate = target_op.create_target_gate([x,y])
            return pygsti.error_generator(tmp_gate, tar_gate, 'pp', 'logGTi')[indices]


        X, Y = np.meshgrid(x, y, indexing='ij')
        Z_linear = np.zeros([nparams, nparams])
        Z_spline = np.zeros([nparams, nparams])
        Z_truth  = np.zeros([nparams, nparams])
        for idx, xx in enumerate(x):
            for idy, yy in enumerate(y):
                Z_linear[idx,idy] = interp_linear(xx,yy)
                Z_spline[idx,idy] = interp_spline(xx,yy)
                Z_truth[idx,idy]  = truth(xx,yy)

        fig = plt.figure(figsize=(10,10))
        ax = plt.axes(projection='3d')

        ax.plot_surface(X, Y, Z_linear-Z_truth, rstride=1, cstride=1,
                        edgecolor='none', alpha=.8)
        ax.plot_surface(X, Y, Z_spline-Z_truth, rstride=1, cstride=1,
                        cmap='viridis', edgecolor='none', alpha=.8)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
    #     ax.set_zlim(-1,1)
        plt.show()

# Build a model from this gate

In [10]:
x_gate = opfactory_spline.create_op([0,np.pi/4])
y_gate = opfactory_spline.create_op([np.pi/2,np.pi/4])

In [11]:
x_gate.from_vector([1.03])
y_gate.from_vector([1.0])
print(np.round_(x_gate,4))
print()
print(np.round_(y_gate,4))

[[ 1.0000  0.0000 -0.0000  0.0000]
 [ 0.0000  1.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.6903  0.7236]
 [ 0.0000  0.0000 -0.7236  0.6903]]

[[ 1.0000  0.0000 -0.0000  0.0000]
 [ 0.0000  0.7071  0.0000 -0.7071]
 [-0.0000 -0.0000  1.0000 -0.0000]
 [ 0.0000  0.7071 -0.0000  0.7071]]


In [12]:
from pygsti.modelpacks import smq1Q_XY

In [13]:
from pathlib import Path
working_dir = Path.cwd()

## Make a fake dataset

In [14]:
#Model only has Gx and Gy gates.  Let's rename them.

model = pygsti.models.ExplicitOpModel([0],'pp')

model['rho0'] = [ 1/np.sqrt(2), 0, 0, 1/np.sqrt(2) ] # density matrix [[1, 0], [0, 0]] in Pauli basis
model['Mdefault'] = pygsti.modelmembers.povms.UnconstrainedPOVM(
    {'0': [ 1/np.sqrt(2), 0, 0, 1/np.sqrt(2) ],   # projector onto [[1, 0], [0, 0]] in Pauli basis
     '1': [ 1/np.sqrt(2), 0, 0, -1/np.sqrt(2) ] }, evotype="default") # projector onto [[0, 0], [0, 1]] in Pauli basis
model['Gxpi2',0] = x_gate
model['Gypi2',0] = y_gate

In [15]:
model.num_params

14

In [16]:
model.to_vector()

array([ 0.7071,  0.0000,  0.0000,  0.7071,  0.7071,  0.0000,  0.0000,  0.7071,  0.7071,  0.0000,  0.0000, -0.7071,
        1.0300,  1.0000])

In [17]:
# Define the error model used to generate data
datagen_model = model.copy()
datagen_params = datagen_model.to_vector()
datagen_params[-2:] = [1.03,1.00]
datagen_model.from_vector(datagen_params)
datagen_model.probabilities( (('Gxpi2',0),('Gypi2',0),))

OutcomeLabelDict([(('0',), 0.7440406663460976), (('1',), 0.2559593336539022)])

In [18]:
model._print_gpindices()

PRINTING MODEL GPINDICES!!!
>>> rho0 [FullState]: 4 params, indices=slice(0, 4, None)
>>> Mdefault [UnconstrainedPOVM]: 8 params, indices=slice(4, 12, None)
>>> Gxpi2:0 [InterpolatedDenseOp]: 1 params, indices=slice(12, 13, None)
>>> Gypi2:0 [InterpolatedDenseOp]: 1 params, indices=slice(13, 14, None)


In [19]:
# # Link the over-rotation errors on Gx and Gy
# model['rho0'].set_gpindices(slice(0,4),model)
# model['Mdefault'].set_gpindices(slice(4,12),model)
# model['Gxpi2',0].set_gpindices(slice(12,13),model)
# model['Gypi2',0].set_gpindices(slice(12,13),model)
# model._rebuild_paramvec()
# model._print_gpindices()
print(model.num_params)

# # Define the error model used to generate data
# datagen_model = model.copy()
# datagen_params = datagen_model.to_vector()
# datagen_params[-1:] = [1.02]
# datagen_model.from_vector(datagen_params)
# datagen_model.probabilities( (('Gxpi2',0),('Gypi2',0),))

14


In [20]:
# Define the perfect target model
target_model = model.copy()
target_params = target_model.to_vector()
target_params[-2:] = [1,1]
# target_model.from_vector(target_params)
target_model.probabilities( (('Gxpi2',0),('Gypi2',0),))

OutcomeLabelDict([(('0',), 0.7440406663460976), (('1',), 0.2559593336539022)])

### Germ and fiducial selection

In [21]:
print(model)
print(model.to_vector())

rho0 = FullState with dimension 4
 0.71   0   0 0.71


Mdefault = UnconstrainedPOVM with effect vectors:
0: FullPOVMEffect with dimension 4
 0.71   0   0 0.71

1: FullPOVMEffect with dimension 4
 0.71   0   0-0.71



Gxpi2:0 = 
InterpolatedDenseOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0 0.69 0.72
   0   0-0.72 0.69


Gypi2:0 = 
InterpolatedDenseOp with shape (4, 4)
 1.00   0   0   0
   0 0.71   0-0.71
   0   0 1.00   0
   0 0.71   0 0.71




[ 0.7071  0.0000  0.0000  0.7071  0.7071  0.0000  0.0000  0.7071  0.7071  0.0000  0.0000 -0.7071  1.0300  1.0000]


In [22]:
pygsti.algorithms.germselection._get_model_params([model])

([<pygsti.models.explicitmodel.ExplicitOpModel at 0x159f8df40>], 2, 0, 2)

In [36]:
final_germs = pygsti.algorithms.germselection.find_germs(
                target_model, randomize=False, force=None, algorithm='greedy', 
                verbosity=4, num_nongauge_params=2)

Using greedy algorithm.
Complete initial germ set succeeds on all input models.
Now searching for best germ set.
Starting germ set optimization. Lower score is better.
  Outer iteration: 1 of 2 amplified, 0 germs
    Inner iter over candidate germs [##################################################] 100.0% Gxpi2:0Gypi2:0Gypi2:0Gypi2:0Gypi2:0Gypi2:0@(0)
    Added Gypi2:0@(0) to final germs (Score: major=-1.0 minor=0.8105694654717684, N: 1)
  Outer iteration: 1 of 2 amplified, 1 germs
    Inner iter over candidate germs [##################################################] 100.0% Gxpi2:0Gypi2:0Gypi2:0Gypi2:0Gypi2:0Gypi2:0@(0)
    Added Gxpi2:0@(0) to final germs (Score: major=-2.0 minor=1.621138935590229, N: 2)
  Outer iteration: 2 of 2 amplified, 2 germs
Constructed germ set:
['Gypi2:0@(0)', 'Gxpi2:0@(0)']
Score: Score: major=0.0 minor=inf, N: 0


In [37]:
final_germs

[Circuit(Gypi2:0@(0)), Circuit(Gxpi2:0@(0))]

In [38]:
fiducial_pairs = pygsti.algorithms.fiducialpairreduction.find_sufficient_fiducial_pairs_per_germ(
                                model, 
                                smq1Q_XY.prep_fiducials([0]),
                                smq1Q_XY.meas_fiducials([0]), 
                                final_germs)

In [39]:
fiducial_pairs

{Circuit(Gypi2:0@(0)): [(0, 0), (0, 1), (0, 2), (1, 1)],
 Circuit(Gxpi2:0@(0)): [(0, 0), (0, 1), (0, 2), (2, 2)]}

In [42]:
fiducial_pairs2 = fiducial_pairs.copy()
for key in fiducial_pairs2.keys():
# key = list(fiducial_pairs2.keys())[0]
    fiducial_pairs2[key] = fiducial_pairs2[key][0:2]

In [43]:
fiducial_pairs2

{Circuit(Gypi2:0@(0)): [(0, 0), (0, 1)],
 Circuit(Gxpi2:0@(0)): [(0, 0), (0, 1)]}

In [44]:
working_dir = Path.cwd() / 'test'

# get experiment design from smq1Q_XY
# exp_design = smq1Q_XY.get_gst_experiment_design(max_max_length=32,qubit_labels=('Q0',))  

# Use fiducial pair reductions
exp_design = pygsti.protocols.StandardGSTDesign(model, 
                                                smq1Q_XY.prep_fiducials([0]), 
                                                smq1Q_XY.meas_fiducials([0]), 
                                                final_germs, 
                                                max_lengths=[1,2,4,8,16,32,64,128,256], 
                                                fiducial_pairs=fiducial_pairs2,
                                                include_lgst=False)

# write an empty data object (creates a template to fill in)
pygsti.io.write_empty_protocol_data(exp_design, working_dir, clobber_ok=True)

# fill in the template with simulated data (you would run the experiment and use actual data)
pygsti.io.fill_in_empty_dataset_with_fake_data(
    datagen_model,
    working_dir / 'data/dataset.txt', num_samples=1000, seed=1234)

# load the data object back in, now with the experimental data
data = pygsti.io.load_data_from_dir(working_dir)

In [45]:
len(data.dataset)

35

## Fisher information matrix

What is the expected covariance in the estimated parameters given the specified experiment? Assuming the GST estimator is unbiased,

$$ \mathrm{cov}\left(\left\langle \vec \theta \right\rangle\right) \le I(\theta)^{-1}$$

Where $I(\vec\theta)$ is the Fisher information matrix, with entries:

$$ I_{m,n} = - \left\langle \frac{\partial^2}{\partial \theta_m \partial \theta_n} \mathrm{log} f(x;\vec\theta) \right\rangle $$

Here $f(x;\vec\theta)$ is the likelihood function. For a single quantum circuit, it is equal to:

$$ f(x;\vec\theta) = \prod_i p_i(\theta)^{x_i} $$

Where $p_i(theta)$ is the probability of measuring the $i$th possible outcome, and $x_i$ is the number of times that outcome was observed. The logarithmic derivatives can be evaluated as:

$$
\begin{align}
I_{m,n} &= - \sum_i \left\langle x_i \frac{\partial^2}{\partial \theta_m \partial \theta_n}  \log p_i(\vec\theta)\right\rangle  \\
        &= - \sum_i \left\langle x_i \frac{\partial}{\partial \theta_m }  \frac{\partial \theta_n p_i(\vec\theta)}{p_i(\vec\theta)}\right\rangle  \\
        &= - \sum_i \left\langle x_i \left(\frac{\partial\theta_m \partial \theta_n p_i(\vec\theta)}{p_i(\vec\theta)} - 
        \frac{(\partial\theta_m p_i(\vec\theta) )(\partial\theta_n p_i(\vec\theta))}{p_i(\vec\theta)^2} \right)
        \right\rangle  \\
I &= N \sum_i \left( 
        \frac{(\nabla p_i(\vec\theta) )(\nabla p_i(\vec\theta))^\mathsf{T}}{p_i(\vec\theta)} -
        \nabla\nabla p_i(\vec\theta) \right) \vert_{\theta = \theta_0}\\
&= N \sum_i \left( 
        \frac{J_i J_i^\mathsf{T}}{p_i} - H_i \right) \vert_{\theta = \theta_0}        
\end{align}
$$



In [46]:
# # circuit -> probabilities
# # what is dp/dtheta?
# # model.sim.dprobs(c) 

# model.sim = 'map'
# model.sim.dprobs(exp_design.all_circuits_needing_data[3])
# hprobs = model.sim.hprobs(exp_design.all_circuits_needing_data[3])
# len(hprobs['0'])
# np.linalg.matrix_rank(hprobs['0'])

# Run GST on the dataset

In [47]:
proto = pygsti.protocols.GateSetTomography(target_model)
results = proto.run(data)

--- Iterative GST: Iter 1 of 9  4 circuits ---: 
  MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
     1 atoms, parameter block size limits (None,)
  *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
      More atom-processors than hosts: each host gets ~1 atom-processors
      Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
  *** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
  --- chi2 GST ---
  Sum of Chi^2 = 0.0112986 (4 data params - 14 (approx) model params = expected mean of -10; p-value = nan)
  Completed in 0.1s
  Iteration 1 took 0.1s
  
--- Iterative GST: Iter 2 of 9  7 circuits ---: 
  MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
     1 atoms, parameter block size limits (None,)
  *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
      More atom-processors than host

In [48]:
# What is the estimated value of the error parameter?

final_model = results.estimates['GateSetTomography'].models['final iteration estimate']
print('Actual: ', datagen_model.to_vector()[-2:])
print('Estimated: ', final_model.to_vector()[-2:])

Actual:  [1.03 1.00]
Estimated:  [1.0304 1.0000]


In [49]:
pprint(np.sqrt(2)*final_model.to_vector()[0:4])
pprint(np.sqrt(2)*final_model.to_vector()[4:8])
pprint(np.sqrt(2)*final_model.to_vector()[8:12])

array([ 1.0001, -0.0730, -0.0239,  0.9976])
array([ 1.0064, -0.0219, -0.0209,  1.0079])
array([ 0.9969,  0.0461,  0.0113, -0.9958])
